In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')

csv_path_cnt = '/content/drive/My Drive/연구실/Olfactory_Connection.csv'
df = pd.read_csv(csv_path_cnt)

door_df = pd.read_csv( '/content/drive/My Drive/연구실/door_imputed_same_perOSN_dist.csv' , index_col = 0)



Mounted at /content/drive


In [ ]:
# ORN id매칭 및 오름차순 정렬
df = pd.read_csv(csv_path_cnt)
df = df[df['directionality'] == 'feedforward'] #feedforward인 거만 추출
orn_ids = set(df[(df['pre_class']=='ORN')& (df['post_class']=='ALPN')]['pre_root_id']) #ORN > ALPN만 추출
orn_ids = sorted(list(orn_ids))
orn_id_idx = {nid:i for i, nid in enumerate(orn_ids)} #각 ORN의 ID와 INDEX 딕셔너리로 구현

In [ ]:
# ORN id와 OR들 딕셔너리로 매칭
conn_df = pd.read_csv('/content/drive/My Drive/연구실/ORN_Receptor_Matching .csv')
orn_to_receptors = {}
for _, row in conn_df.iterrows(): #conn_df 의 데이터에서 각행마다의 뉴런데이터
    orn_id = row['root_id']
    receptor = row['receptor']
    if orn_id not in orn_to_receptors:  #중복방지
        orn_to_receptors[orn_id] = []
    orn_to_receptors[orn_id].append(receptor)

In [ ]:
# 입력 데이터 만들기
door_df = pd.read_csv( '/content/drive/MyDrive/연구실/door_imputed_same_perOSN_dist.csv' , index_col = 0)
num_odor = len(door_df)
num_orn = len(orn_id_idx)
in_data = np.zeros((num_odor, num_orn))

for orn_id, orn_i in orn_id_idx.items():
    receptor_list = orn_to_receptors.get(orn_id, [])
    valid_receptors = [r for r in receptor_list if r in door_df.columns]

    if valid_receptors:
        # door_df에서 해당 receptor들만 골라서 더함 → Series로 반환됨
        in_data[:, orn_i] = door_df[valid_receptors].sum(axis=1).values

print(in_data.shape)

# Convert numpy array to pandas DataFrame
in_data_df = pd.DataFrame(in_data, index=door_df.index, columns=orn_id_idx.keys())

in_data_df.to_csv('/content/drive/MyDrive/연구실/DoOR_data.csv', index=True)

(250, 2278)
